In [5]:
import pandas as pd

# Load your data
df = pd.read_csv("Ruralurbancontinuumcodes2023.csv", encoding="latin1")

# Filter out 'Description' rows
df_filtered = df[df["Attribute"] != "Description"]

# Pivot so each county is a row and attributes are columns
df_pivot = df_filtered.pivot_table(
    index=["FIPS", "State", "County_Name"],
    columns="Attribute",
    values="Value",
    aggfunc="first"  # in case there's any duplicate, it takes the first
).reset_index()

# Optional: flatten column names if needed
df_pivot.columns.name = None

# get dtypes for columns
df_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3235 entries, 0 to 3234
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   FIPS             3235 non-null   int64 
 1   State            3235 non-null   object
 2   County_Name      3235 non-null   object
 3   Population_2020  3235 non-null   object
 4   RUCC_2023        3233 non-null   object
dtypes: int64(1), object(4)
memory usage: 126.5+ KB


In [6]:
# convert columns to numeric
df_pivot['RUCC_2023'] = pd.to_numeric(df_pivot['RUCC_2023'], errors='coerce')
df_pivot['Population_2020'] = pd.to_numeric(df_pivot['Population_2020'], errors='coerce')

In [9]:
# strip county and parish from County_Name
# strip whitespace
df_pivot['County_Name'] = df_pivot['County_Name'].str.replace(r'\s+(County|Parish)$', '', regex=True).str.strip()
df_pivot['county_state'] = (df_pivot['County_Name'] + ', ' + df_pivot['State']).str.lower()




In [10]:
#read in 2025 dialysis clinic csv

df2025=pd.read_csv('df2025_full_census.csv')

# clean df2025 like df_pivot
# Do the same cleaning to match formatting
df2025['County/Parish'] = df2025['County/Parish'].str.replace(r'\s+(County|Parish)$', '', regex=True).str.strip()
df2025['county_state'] = (df2025['County/Parish'] + ', ' + df2025['State']).str.lower()


In [11]:
# convert datatypes to int in df_pivot

df_pivot['Population_2020'] = pd.to_numeric(df_pivot['Population_2020'], errors='coerce').astype('Int64')
df_pivot['RUCC_2023'] = pd.to_numeric(df_pivot['RUCC_2023'], errors='coerce').astype('Int64')


In [7]:
# save to csv

df_pivot.to_csv("pivoted_county_data.csv", index=False)

In [ ]:
# merge df_pivot and df2025

df2025 = df2025.merge(
    df_pivot,
    on='county_state',
    how='left'
)

df2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7556 entries, 0 to 7555
Columns: 153 entries, CMS Certification Number (CCN) to RUCC_2023
dtypes: Int64(2), float64(80), int64(27), object(44)
memory usage: 8.8+ MB


In [14]:
# verify
df2025.head(10)

,CMS Certification Number (CCN),Network,Facility Name,Five Star Date,Five Star,Five Star Data Availability Code,Address Line 1,Address Line 2,City/Town,State_x,...,full_address,latitude,longitude,census_tract,county_state,FIPS,State_y,County_Name,Population_2020,RUCC_2023
0,12306,8,CHILDRENS HOSPITAL DIALYSIS,01Jan2020-31Dec2023,NaN,260,1600 7TH AVENUE SOUTH,NaN,BIRMINGHAM,AL,...,"1600 7TH AVENUE SOUTH, BIRMINGHAM, AL 35233",33.504018,-86.805215,1.073005e+09,"jefferson, al",1073.0,AL,Jefferson,674721,1
1,12500,8,FMC CAPITOL CITY,01Jan2020-31Dec2023,1.0,1,255 S JACKSON STREET,NaN,MONTGOMERY,AL,...,"255 S JACKSON STREET, MONTGOMERY, AL 36104",32.374048,-86.295939,1.101001e+09,"montgomery, al",1101.0,AL,Montgomery,228954,2
2,12501,8,DaVita Gadsden Dialysis,01Jan2020-31Dec2023,2.0,1,409 SOUTH FIRST STREET,NaN,GADSDEN,AL,...,"409 SOUTH FIRST STREET, GADSDEN, AL 35901",34.006750,-86.003132,1.055001e+09,"etowah, al",1055.0,AL,Etowah,103436,3
3,12502,8,DaVita Tuscaloosa University Dialysis,01Jan2020-31Dec2023,1.0,1,220 15TH STREET,NaN,TUSCALOOSA,AL,...,"220 15TH STREET, TUSCALOOSA, AL 35401",33.198990,-87.538230,1.125012e+09,"tuscaloosa, al",1125.0,AL,Tuscaloosa,227036,2
4,12505,8,DaVita PDI-Montgomery,01Jan2020-31Dec2023,3.0,1,1001 FOREST AVENUE,NaN,MONTGOMERY,AL,...,"1001 FOREST AVENUE, MONTGOMERY, AL 36106",32.369331,-86.285969,1.101001e+09,"montgomery, al",1101.0,AL,Montgomery,228954,2
5,12506,8,DaVita Dothan Dialysis,01Jan2020-31Dec2023,3.0,1,216 GRACELAND DR.,NaN,DOTHAN,AL,...,"216 GRACELAND DR., DOTHAN, AL 36305",31.231700,-85.452660,1.069040e+09,"houston, al",1069.0,AL,Houston,107202,3
6,12507,8,FMC MOBILE,01Jan2020-31Dec2023,3.0,1,2620 OLD SHELL RD,NaN,MOBILE,AL,...,"2620 OLD SHELL RD, MOBILE, AL 36607",30.692555,-88.102452,1.097003e+09,"mobile, al",1097.0,AL,Mobile,414809,2
7,12508,8,DaVita Birmingham East Dialysis,01Jan2020-31Dec2023,2.0,1,1105 EAST PARK DRIVE,NaN,BIRMINGHAM,AL,...,"1105 EAST PARK DRIVE, BIRMINGHAM, AL 35235",33.587540,-86.680430,1.073006e+09,"jefferson, al",1073.0,AL,Jefferson,674721,1
8,12512,8,FMC SELMA,01Jan2020-31Dec2023,1.0,1,905 MEDICAL CENTER PARKWAY,NaN,SELMA,AL,...,"905 MEDICAL CENTER PARKWAY, SELMA, AL 36701",32.427473,-87.052770,1.047957e+09,"dallas, al",1047.0,AL,Dallas,38462,4
9,12513,8,BMA LANGDALE,01Jan2020-31Dec2023,5.0,1,8 MEDICAL PARK,NaN,VALLEY,AL,...,"8 MEDICAL PARK, VALLEY, AL 36854",32.828959,-85.168810,1.017955e+09,"chambers, al",1017.0,AL,Chambers,34772,6


In [31]:
# convert fips to string

# Step 1: Convert to int (if it’s float)
df2025['FIPS'] = df2025['FIPS'].astype('Int64')  # allows for NaNs

# Step 2: Convert to string and zero-pad
df2025['FIPS'] = df2025['FIPS'].astype(str).str.zfill(5)

df2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7556 entries, 0 to 7555
Columns: 153 entries, CMS Certification Number (CCN) to RUCC_2023
dtypes: Int64(2), float64(79), int64(27), object(45)
memory usage: 8.8+ MB


In [32]:
# make FIPS all lowercase

df2025.columns = df2025.columns.str.lower()

In [33]:
#calculate clinics per 100k population

# Step 1: Count number of clinics per county
clinic_counts = df2025.groupby('county_state').size().reset_index(name='clinic_count')

# Step 2: Get unique population per county from df2025 (assuming it's already merged in)
population = df2025[['county_state', 'population_2020']].drop_duplicates()

# Step 3: Merge counts with population
clinic_stats = clinic_counts.merge(population, on='county_state', how='left')

# Step 4: Calculate clinics per 100,000 residents
clinic_stats['clinics_per_100k'] = (clinic_stats['clinic_count'] / clinic_stats['population_2020']) * 100000


In [34]:
clinic_stats.to_csv('clinic_stats_by_county.csv', index=False)


In [35]:
df2025[['county_state', 'fips']].drop_duplicates().head()

,county_state,fips
0,"jefferson, al",01073
1,"montgomery, al",01101
2,"etowah, al",01055
3,"tuscaloosa, al",01125
5,"houston, al",01069


In [39]:
# convert county_state to FIPS code for mapping in QGIS

# Load the FIPS reference table
fips_df = pd.read_csv("https://raw.githubusercontent.com/kjhealy/fips-codes/master/state_and_county_fips_master.csv")

# Preview column names
print(fips_df.columns)


# For the fips_df dataframe
fips_df['county_state'] = (
    fips_df['name'].str.replace(' County| Parish', '', regex=True)
    .str.strip()
    .str.title()
    + ', ' + fips_df['state'].str.upper()
)


# Create a matching county_state field
fips_df['county_state'] = fips_df['name'].str.replace(' County| Parish', '', regex=True).str.strip() + ', ' + fips_df['state']
fips_df['fips'] = fips_df['fips'].astype(str).str.zfill(5)

fips_df.head(20)

Index(['fips', 'name', 'state'], dtype='object')


,fips,name,state,county_state
0,00000,UNITED STATES,NaN,NaN
1,01000,ALABAMA,NaN,NaN
2,01001,Autauga County,AL,"Autauga, AL"
3,01003,Baldwin County,AL,"Baldwin, AL"
4,01005,Barbour County,AL,"Barbour, AL"
5,01007,Bibb County,AL,"Bibb, AL"
6,01009,Blount County,AL,"Blount, AL"
7,01011,Bullock County,AL,"Bullock, AL"
8,01013,Butler County,AL,"Butler, AL"
9,01015,Calhoun County,AL,"Calhoun, AL"


In [ ]:
# drop NaN entries
fips_df['county_state'] = fips_df['county_state'].str.strip()
fips_df = fips_df[fips_df['county_state'] != '']

fips_df = fips_df.dropna(subset=['county_state'])
fips_df.head(10)

,fips,name,state,county_state
2,01001,Autauga County,AL,"Autauga, AL"
3,01003,Baldwin County,AL,"Baldwin, AL"
4,01005,Barbour County,AL,"Barbour, AL"
5,01007,Bibb County,AL,"Bibb, AL"
6,01009,Blount County,AL,"Blount, AL"
7,01011,Bullock County,AL,"Bullock, AL"
8,01013,Butler County,AL,"Butler, AL"
9,01015,Calhoun County,AL,"Calhoun, AL"
10,01017,Chambers County,AL,"Chambers, AL"
11,01019,Cherokee County,AL,"Cherokee, AL"


In [55]:
#make county_state lower case in fips_df
fips_df['county_state'] = fips_df['county_state'].str.lower()
fips_df.head(10)
fips_df['fips'] = fips_df['fips'].str.strip()

In [56]:
fips_df['fips'].info

<bound method Series.info of 2       01001
3       01003
4       01005
5       01007
6       01009
        ...  
3190    56037
3191    56039
3192    56041
3193    56043
3194    56045
Name: fips, Length: 3143, dtype: object>

In [68]:
clinic_county=pd.read_csv('clinic_stats_by_county.csv')
clinic_county.head(5)



,county_state,clinic_count,population_2020,clinics_per_100k
0,"acadia, la",1,57576.0,1.736835
1,"accomack, va",1,33413.0,2.992847
2,"ada, id",5,494967.0,1.010168
3,"adair, mo",1,25314.0,3.950383
4,"adair, ok",1,19495.0,5.129520


In [69]:
# merge the two dataframs to add fips data to clinic_stats

clinic_county = clinic_county.merge(
    fips_df[['county_state', 'fips']],
    on='county_state',
    how='left'
)


clinic_county.head()

,county_state,clinic_count,population_2020,clinics_per_100k,fips
0,"acadia, la",1,57576.0,1.736835,22001
1,"accomack, va",1,33413.0,2.992847,51001
2,"ada, id",5,494967.0,1.010168,16001
3,"adair, mo",1,25314.0,3.950383,29001
4,"adair, ok",1,19495.0,5.129520,40001


In [ ]:
# ensure fips has five digits, preserve leading 0s

clinic_county['fips'] = clinic_county['fips'].astype(str).str.zfill(5)
clinic_county['fips'].head(50)

0     22001
1     51001
2     16001
3     29001
4     40001
5     08001
6     17001
7     18001
8     28001
9     31001
10    39001
11    42001
12    53001
13    00nan
14    00nan
15    45003
16    27001
17    12001
18    37001
19    06001
20    08003
21    36001
22    56001
23    51003
24    28003
25    37003
26    51510
27    26005
28    24001
29    42003
30    18003
31    39003
32    45005
33    26007
34    06005
35    51007
36    00nan
37    51009
38    00nan
39    45007
40    47001
41    48001
42    23001
43    48005
44    24003
45    27003
46    37007
47    04001
48    13001
49    51011
Name: fips, dtype: object

In [71]:
clinic_county['fips'] = clinic_county['fips'].astype(str).str.zfill(5)

In [72]:
# export as csv

clinic_county.to_csv("fips_clinic_final.csv", index=False)
